In [1]:
# !pip3 install --upgrade pip
# !pip3 install --upgrade pandas
# !pip3 install --upgrade sklearn

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
pd.options.mode.chained_assignment = None 

dataset = pd.read_csv('Kraków 2021-02-01 to 2022-01-26.csv')
dataset

,name,datetime,temp,feelslike,dew,humidity,precip,precipprob,preciptype,snow,...,sealevelpressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,severerisk,conditions,icon,stations
0,Kraków,2021-02-01T00:00:00,-10.3,-15.5,-12.1,86.35,0.00,NaN,NaN,NaN,...,1008.5,9.2,5.9,NaN,NaN,NaN,NaN,Clear,clear-night,"12575099999,12566099999,12660099999,1191809999..."
1,Kraków,2021-02-01T01:00:00,-12.2,-17.9,-13.9,86.77,0.00,NaN,NaN,NaN,...,1008.2,9.2,5.0,NaN,NaN,NaN,NaN,Clear,clear-night,"12575099999,12566099999,12660099999,1191809999..."
2,Kraków,2021-02-01T02:00:00,-12.2,-18.2,-14.1,86.15,0.00,NaN,NaN,NaN,...,1007.7,0.0,5.0,NaN,NaN,NaN,NaN,Clear,clear-night,"12575099999,12566099999,12660099999,1191809999..."
3,Kraków,2021-02-01T03:00:00,-12.5,-18.1,-14.3,86.12,0.00,NaN,NaN,NaN,...,1007.5,0.0,4.9,NaN,NaN,NaN,NaN,Clear,clear-night,"12575099999,12566099999,12660099999,1191809999..."
4,Kraków,2021-02-01T04:00:00,-12.1,-18.8,-13.2,90.91,0.00,NaN,NaN,NaN,...,1006.9,0.0,5.0,NaN,NaN,NaN,NaN,Clear,clear-night,"12575099999,12566099999,12660099999,1191809999..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8635,Kraków,2022-01-26T19:00:00,0.9,-3.0,0.1,94.55,0.23,100.0,"rain,snow",0.0,...,1024.2,98.2,4.3,2.0,0.0,0.0,10.0,"Snow, Rain, Overcast",rain,"12575099999,12566099999,12660099999,1191809999..."
8636,Kraków,2022-01-26T20:00:00,1.1,-4.5,0.3,93.99,0.00,0.0,NaN,0.0,...,1024.2,91.8,14.4,2.0,0.0,0.0,10.0,Overcast,cloudy,"12575099999,12566099999,12660099999,1191809999..."
8637,Kraków,2022-01-26T21:00:00,1.1,-3.9,0.0,92.53,0.00,0.0,NaN,0.0,...,1024.0,91.8,12.2,2.0,0.0,0.0,10.0,Overcast,cloudy,"12575099999,12566099999,12660099999,1191809999..."
8638,Kraków,2022-01-26T22:00:00,1.1,-3.5,0.0,92.42,0.00,0.0,snow,0.0,...,1024.1,91.0,12.1,2.0,0.0,0.0,10.0,Overcast,cloudy,"12575099999,12566099999,12660099999,1191809999..."


In [4]:
columns_names = [col for col in dataset]

# pd.DataFrame(columns_names, columns=['column_name'])

In [5]:
selected_columns = [2, 5, 21]
selected_columns_names = [columns_names[col] for col in selected_columns]
print(selected_columns_names)

dataset = dataset.dropna(subset=selected_columns_names)
dataset=dataset.reset_index(drop=True)
training_set_wlabels = dataset.iloc[:,selected_columns]


['temp', 'humidity', 'conditions']


In [6]:
dataset = dataset.dropna(subset=selected_columns_names)
dataset=dataset.reset_index(drop=True)
training_set_wlabels = dataset.iloc[:,selected_columns]


conditions_guard = True if 'conditions' in selected_columns_names else False
if conditions_guard:
#     cond = set()
#     [cond.add(it) for it in training_set_wlabels.loc[:, 'conditions']]
#     cond = list(cond)
    cond = ['Snow', 'Rain, Partially cloudy', 'Snow, Partially cloudy', 'Rain', 'Partially cloudy', 'Rain, Overcast', 'Snow, Rain, Overcast', 'Overcast', 'Snow, Overcast', 'Clear', 'Snow, Rain, Partially cloudy']
    for label in cond:
        training_set_wlabels.loc[:, label] = 0
        training_set_wlabels.loc[training_set_wlabels['conditions']==label, label] = 1
    training_set_wlabels = training_set_wlabels.drop(columns='conditions')


training_set = training_set_wlabels.values
# training_set

In [7]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range=(0,1))
training_set_scaled = sc.fit_transform(training_set)

In [20]:
import joblib
joblib.dump(sc, "scaler.save")

['scaler.save']

In [8]:
x_train = []
y_train = []
n_future = 12 # next 4 hours weather forecast
n_past = 48 # Past 30 hours 
for i in range(0,len(training_set_scaled)-n_past-n_future+1):
    x_train.append(training_set_scaled[i : i + n_past])     
    y_train.append(training_set_scaled[i + n_past : i + n_past + n_future])
x_train , y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0] , x_train.shape[2], x_train.shape[1]) )
y_train = np.reshape(y_train, (y_train.shape[0] , y_train.shape[2], y_train.shape[1]) )

In [9]:
# print("X")
# print(x_train.shape)
# print(x_train[0])
# print("Y")
# print(y_train.shape)
# print(y_train[0])

In [10]:
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Dropout, Bidirectional

In [11]:
# from keras.models import load_model
# model = load_model('weather_forecast_model')

In [12]:
model = Sequential()
model.add(Bidirectional(LSTM(units=30, return_sequences=True, input_shape = (x_train.shape[1],x_train.shape[2]) ) ))
model.add(Dropout(0.2))
model.add(LSTM(units= 30 , return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units= 30 , return_sequences=True))
model.add(Dropout(0.2))
model.add(Dense(units = n_future,activation='linear'))
model.compile(optimizer='adam', loss='mean_squared_error',metrics=['acc'])

from time import time
tstart = time()
model.fit(x_train, y_train, epochs=500,batch_size=32)
tstop = time()

Epoch 1/500
269/269 [==============================] - 15s 21ms/step - loss: 0.1038 - acc: 0.0548
Epoch 2/500
269/269 [==============================] - 6s 21ms/step - loss: 0.0695 - acc: 0.0982
Epoch 3/500
269/269 [==============================] - 6s 21ms/step - loss: 0.0594 - acc: 0.1260
Epoch 4/500
269/269 [==============================] - 6s 21ms/step - loss: 0.0556 - acc: 0.1593
Epoch 5/500
269/269 [==============================] - 6s 21ms/step - loss: 0.0535 - acc: 0.1843
Epoch 6/500
269/269 [==============================] - 6s 21ms/step - loss: 0.0520 - acc: 0.2048
Epoch 7/500
269/269 [==============================] - 6s 21ms/step - loss: 0.0507 - acc: 0.2241
Epoch 8/500
269/269 [==============================] - 6s 23ms/step - loss: 0.0499 - acc: 0.2442
Epoch 9/500
269/269 [==============================] - 6s 21ms/step - loss: 0.0490 - acc: 0.2599
Epoch 10/500
269/269 [==============================] - 5s 20ms/step - loss: 0.0484 - acc: 0.2756
Epoch 11/500
269/269 [======

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 13, 60)           18960     
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 13, 60)            0         
                                                                 
 lstm_1 (LSTM)               (None, 13, 30)            10920     
                                                                 
 dropout_1 (Dropout)         (None, 13, 30)            0         
                                                                 
 lstm_2 (LSTM)               (None, 13, 30)            7320      
                                                                 
 dropout_2 (Dropout)         (None, 13, 30)            0         
                                                        

In [14]:
# for layer in model.layers:
#     print(layer.input_shape)

# print('-------')

# for layer in model.layers:
#     print(layer.output_shape)

In [15]:
print(f"time passed: {(int)(tstop-tstart)//3600}h {(int)(tstop-tstart)%3600//60}m {(int)(tstop-tstart)%60}s")

time passed: 0h 46m 56s


In [16]:
model.save('weather_forecast_model')

INFO:tensorflow:Assets written to: weather_forecast_model/assets


INFO:tensorflow:Assets written to: weather_forecast_model/assets


In [21]:
!tar czf weather_forecast_model.tgz weather_forecast_model

In [69]:
testdata = pd.read_csv('Kraków 2022-03-01 to 2022-03-09.csv')

offset = 0

testdataset_wlabels = testdata.iloc[offset:offset+30,selected_columns]

if conditions_guard:
    for label in cond:
        testdataset_wlabels[label] = 0
        testdataset_wlabels.loc[testdataset_wlabels['conditions']==label, label] = 1
    testdataset_wlabels = testdataset_wlabels.drop(columns='conditions')

testdataset = testdataset_wlabels.values
testing = sc.transform(testdataset)
testing = np.array(testing)
# print(testing.shape)
testing = np.reshape(testing,(1,testing.shape[1],testing.shape[0]))
# print(testing.shape)

In [80]:
predicted_weather = model.predict(testing)
# print(predicted_weather)
# print(predicted_weather.shape)
predicted_weather = np.reshape(predicted_weather, (predicted_weather.shape[2],predicted_weather.shape[1]))
# print(predicted_weather.shape)
predicted_weather = sc.inverse_transform(predicted_weather)

In [81]:
real_weather = pd.read_csv('Kraków 2022-03-01 to 2022-03-09.csv')
real_weather = real_weather.iloc[offset+30:offset+34,selected_columns]

if conditions_guard:
    predicted_weather_wlabels = pd.DataFrame(predicted_weather, columns=[col for col in selected_columns_names if col != 'conditions']+cond)
    max_cond = np.argmax(predicted_weather[:, 2:], axis=1)
    for i, mcond in enumerate(max_cond):
        predicted_weather_wlabels.loc[i, 'conditions'] = cond[mcond]
    predicted_weather_wlabels = predicted_weather_wlabels.drop(columns=cond)
else:
    predicted_weather_wlabels = pd.DataFrame(predicted_weather, columns=selected_columns_names)

In [82]:
real_weather

,temp,windspeed,conditions
30,-5.7,3.7,Clear
31,-5.9,3.6,Clear
32,-5.1,1.9,Clear
33,-2.8,2.7,Partially cloudy


In [83]:
predicted_weather_wlabels

,temp,windspeed,conditions
0,2.797085,4.985973,Clear
1,5.782664,5.140410,Clear
2,-0.249886,6.630127,Clear
3,-1.368810,6.353525,Clear


In [4]:
# import pandas as pd
# df = pd.concat(
#     map(pd.read_csv, ['Kraków 2021-02-01 to 2021-09-25.csv', 'Kraków 2021-09-26 to 2021-11-05.csv', 'Kraków 2021-11-06 to 2021-12-16.csv', 'Kraków 2021-12-17 to 2022-01-26.csv']), ignore_index=True)
# df.to_csv('Kraków 2021-02-01 to 2022-01-26.csv', header=True, index=False)

In [5]:
# pd.read_csv('Kraków 2021-02-01 to 2022-01-26.csv').head()

,name,datetime,temp,feelslike,dew,humidity,precip,precipprob,preciptype,snow,...,sealevelpressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,severerisk,conditions,icon,stations
0,Kraków,2021-02-01T00:00:00,-10.3,-15.5,-12.1,86.35,0.0,NaN,NaN,NaN,...,1008.5,9.2,5.9,NaN,NaN,NaN,NaN,Clear,clear-night,"12575099999,12566099999,12660099999,1191809999..."
1,Kraków,2021-02-01T01:00:00,-12.2,-17.9,-13.9,86.77,0.0,NaN,NaN,NaN,...,1008.2,9.2,5.0,NaN,NaN,NaN,NaN,Clear,clear-night,"12575099999,12566099999,12660099999,1191809999..."
2,Kraków,2021-02-01T02:00:00,-12.2,-18.2,-14.1,86.15,0.0,NaN,NaN,NaN,...,1007.7,0.0,5.0,NaN,NaN,NaN,NaN,Clear,clear-night,"12575099999,12566099999,12660099999,1191809999..."
3,Kraków,2021-02-01T03:00:00,-12.5,-18.1,-14.3,86.12,0.0,NaN,NaN,NaN,...,1007.5,0.0,4.9,NaN,NaN,NaN,NaN,Clear,clear-night,"12575099999,12566099999,12660099999,1191809999..."
4,Kraków,2021-02-01T04:00:00,-12.1,-18.8,-13.2,90.91,0.0,NaN,NaN,NaN,...,1006.9,0.0,5.0,NaN,NaN,NaN,NaN,Clear,clear-night,"12575099999,12566099999,12660099999,1191809999..."
